In [ ]:
from pathlib import Path
import datetime

import matplotlib.pyplot as plt
from matplotlib import gridspec
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from bs4 import BeautifulSoup

import stock

In [ ]:
def get_simulation_results(
    watch_list: list[str], target_date: datetime.date = datetime.date.today()
):
    results = []
    for idx, code in enumerate(watch_list):
        stop_condition = stock.simulation.OnielStopCondition()
        res = stock.simulation.run(code, target_date, stop_condition)
        if res.buying_price > 0:
            results.append(
                {"code": code, "duration": res.duration, "profit": round(res.profit * 100)}
            )
    return results


In [ ]:
original_date = datetime.date(year=2022, month=6, day=8)
for i in range(0, 365, 5):
    target_date = original_date + datetime.timedelta(days=i)
    watch_list = stock.trend_template.get_watch_list_v3(target_date)
    results = get_simulation_results(watch_list, target_date)
    success = [res for res in results if res["profit"] == 20]
    print("date = {}, success / total = {} / {}".format(target_date, len(success), len(results)))


In [ ]:
stock.kabutan.data.get_number_of_shares("8142")

In [ ]:
code = "8142"
base_url = "https://kabutan.jp/stock/?code={}"
res = requests.get(base_url.format(code))
soup = BeautifulSoup(res.text, features="lxml")

In [ ]:
for table in soup.find("div", {"id": "kobetsu_left"}).findAll("table"):
    for table_row in table.find_all("tr"):
        th = table_row.find("th")
        if th is not None and th.text == "発行済株式数":
            td = table_row.find("td")
            print(td.text)
            #convert_to_number(td.text)
            break
    

In [ ]:
stock.util.convert_to_number(td.text)

In [ ]:
code_list = stock.get_code_list()
target_date = datetime.date(2022, 6, 8)

stop_condition =stock.simulation.OnielStopCondition()
results = [stock.simulation.run(code, target_date, stop_condition) for code in code_list]

In [ ]:
res = stock.simulation.run("5032", target_date, stop_condition)

In [ ]:
res

In [ ]:
profit_results = [res for res in results if round(res.profit * 100) >= 20]

In [ ]:
len(profit_results)

In [ ]:
# 新高値銘柄のバックテスト
code_df = stock.get_code_df()
code_list = code_df["コード"].to_list()

target_date = datetime.date(year=2023, month=6, day=1)
highest_codes = []
for code in code_list:
    df = stock.kabutan.read_data_csv(stock.DATA_DIR / "daily" / f"{code}.csv")
    df = df.filter((pl.col("date") <= target_date) & (pl.col("date") >= target_date - datetime.timedelta(days=365)))
    if len(df) == 0:
        continue
    if df["high"].max() * 0.998 < df["high"][-1]:
        highest_codes.append(code)
    break